In [1]:
#taking input from the train and test data file
import pandas as pd
from sklearn.preprocessing import LabelEncoder
pd.options.mode.chained_assignment = None
train=2500000
dataset = pd.read_csv("train.csv",sep=',',nrows=train)
file1 = open("test.csv", "r")
dt1 = pd.read_csv(file1, sep = ",")
dataset.shape


(2500000, 8)

In [2]:
dataset.shape

(2500000, 8)

In [3]:
dt1.shape

(9914, 7)

In [4]:
#checking the shape of the train dataset
print("Current shape of dataset :",dataset.shape)
dataset.head(4)

Current shape of dataset : (2500000, 8)


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1


In [5]:
dataset.iloc[0,:]

key                  2009-06-15 17:26:21.0000001
fare_amount                                  4.5
pickup_datetime          2009-06-15 17:26:21 UTC
pickup_longitude                      -73.844311
pickup_latitude                        40.721319
dropoff_longitude                      -73.84161
dropoff_latitude                       40.712278
passenger_count                                1
Name: 0, dtype: object

In [6]:
dt1.iloc[0,:]

key                  2015-01-27 13:08:24.0000002
pickup_datetime          2015-01-27 13:08:24 UTC
pickup_longitude                       -73.97332
pickup_latitude                        40.763805
dropoff_longitude                      -73.98143
dropoff_latitude                       40.743835
passenger_count                                1
Name: 0, dtype: object

In [7]:
print("Current shape of train dataset :",dataset.shape)
print("Current shape of test dataset :",dt1.shape)

Current shape of train dataset : (2500000, 8)
Current shape of test dataset : (9914, 7)


In [8]:
Z=dt1.loc[:,'key']

In [9]:
#separating out the fare_amount column from train data set
var_columns=[c for c in dataset.columns if c not in ['fare_amount']]
Y=dataset.loc[:,'fare_amount']
dataset=dataset.loc[:,var_columns]
print("Current shape of dataset :",dataset.shape)
print("Current shape of dt1 :",dt1.shape)

Current shape of dataset : (2500000, 7)
Current shape of dt1 : (9914, 7)


In [10]:
#concatenating both the data sets so as to have common label encoder values for data under same columns of both test and train data sets
dt2=pd.concat([dataset, dt1], axis=0)

In [11]:
#converting the given attribute to different attributes which include year,month,date,day,hour,minute etc and also adding 2 other attributes which are 'abs_diff_longitude' and 'abs_diff_latitude'
dt2['pickup_datetime'].head(1)
dt2['pickup_datetime']=pd.to_datetime(dt2['pickup_datetime'], infer_datetime_format=True)
dt2['year'] = dt2['pickup_datetime'].dt.year
dt2['Month'] = dt2['pickup_datetime'].dt.month
dt2['Date'] = dt2['pickup_datetime'].dt.day
dt2['Day of Week'] = dt2['pickup_datetime'].dt.dayofweek
dt2['Hour'] = dt2['pickup_datetime'].dt.hour
dt2['Minute'] = dt2['pickup_datetime'].dt.minute
dt2['abs_diff_longitude'] = (dt2.dropoff_longitude - dt2.pickup_longitude)
dt2['abs_diff_latitude'] = (dt2.dropoff_latitude - dt2.pickup_latitude)
dt2=dt2.drop(['key', 'pickup_datetime'], axis = 1)

In [26]:
dt2.isnull().sum()

pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
passenger_count       0
year                  0
Month                 0
Date                  0
Day of Week           0
Hour                  0
Minute                0
abs_diff_longitude    0
abs_diff_latitude     0
distance              0
dtype: int64

In [12]:
#feeatures for which Nan values are filled by frequency
features11=['dropoff_longitude','dropoff_latitude','abs_diff_longitude','abs_diff_latitude']
for i in range(len(features11)):
    dt2[features11[i]] = dt2[features11[i]].fillna(dt2[features11[i]].mode().iloc[0])

In [13]:

from math import radians, cos, sin, asin, sqrt
def Distance(parameter):
    L1=parameter[0]
    L2=parameter[1]
    Dlon=parameter[2]
    Dlat=parameter[3]
    # convert decimal degrees to radians 
    L1,L2,Dlon,Dlat = map(radians, [L1,L2,Dlon,Dlat])
    # haversine formula 
    dist = sin(Dlat/2)**2 + cos(L1) * cos(L2) * sin(Dlon/2)**2
    dist=  2 * asin(sqrt(dist)) 
    # Radius of earth in kilometers is 6378.1
    dist= 6378.1*dist
    return dist
dt2['distance']=dt2[['pickup_latitude','dropoff_latitude','abs_diff_longitude','abs_diff_latitude']].apply(Distance,axis=1)

In [15]:
dataset = dt2.iloc[:train,:]
dt1 = dt2.iloc[train:,:]

In [16]:
#test data
var_columns1=[c for c in dt1.columns]
X_test=dt1.loc[:,var_columns1]

In [18]:
#considering validation where 90% data used for training and 10% for validation from train data
from sklearn.model_selection import train_test_split
X_train,X_valid,Y_train,Y_valid=train_test_split(dataset,Y,test_size=0.1,random_state=42)
X_train.shape,X_valid.shape,Y_train.shape,Y_valid.shape

((2250000, 14), (250000, 14), (2250000,), (250000,))

In [19]:
Y_train

1317330    10.1
1457476    11.5
1280011     5.3
1444750     5.3
1174345     6.1
           ... 
110268     10.5
1692743     4.5
2356330     8.0
2229084     5.0
2219110     6.1
Name: fare_amount, Length: 2250000, dtype: float64

In [20]:
##2nd best model according to RMSE:3.46
##XG boost regressor
#import xgboost
#model=xgboost.XGBRegressor(n_estimators=300,max_depth=15,max_leaves=9,random_state=0)
#model.fit(X_train,Y_train)
#y_test_pred=Regression_xgbooost.predict(X_test)
#print(y_test_pred)

In [21]:
#best model according the RMSE :3.20
#Random Forest classifier
from sklearn.ensemble import RandomForestRegressor
model=RandomForestRegressor(n_estimators=300,random_state=0,max_depth=15)
model.fit(X_train,Y_train)

RandomForestRegressor(max_depth=15, n_estimators=300, random_state=0)

In [22]:
#finding the predicted values for both test and validation sets
from sklearn.metrics import roc_auc_score
Y_train_pred = model.predict(X_train)
Y_valid_pred = model.predict(X_valid)
Y_test_pred=model.predict(X_test)

In [23]:
#calculation of RMSE for validation set of size 250000
import numpy as np
from sklearn.metrics import mean_squared_error
print(np.sqrt(mean_squared_error(Y_valid,Y_valid_pred)))

4.2645606218938426


In [24]:
Y_test_pred.shape
# #Output file to upload on Kaggle
k=[]
for i in range(9914):
     k.append(i)
k=np.array(k)
my_submission = pd.DataFrame({'key': Z, 'fare_amount': Y_test_pred})
# # you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)